# Train SMel model

In [18]:
%load_ext autoreload
%autoreload 2

from keras import optimizers
from keras.callbacks import CSVLogger
import numpy as np
import os
import glob
import sys

sys.path.insert(0,'../..')
from sed_endtoend.smel.model import SMel
from sed_endtoend.data_generator import DataGenerator, Scaler
from sed_endtoend.gen_mel_filters import mel_filters

from params import *

os.environ["CUDA_VISIBLE_DEVICES"]="1"

# files parameters
Nfiles = None
resume = False
load_subset = Nfiles

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
params = {'sequence_time': sequence_time, 'sequence_hop_time':sequence_hop_time,
          'label_list':label_list,'audio_hop':audio_hop, 'audio_win':audio_win,
          'n_fft':n_fft,'sr':sr,'mel_bands':mel_bands,'normalize':normalize_data, 
          'frames':frames,'get_annotations':get_annotations, 'dataset': dataset}

sequence_frames = int(np.ceil(sequence_time*sr/audio_hop))

# Datasets
labels = {}# Labels

train_files = sorted(glob.glob(os.path.join(audio_folder,'train', '*.wav')))
val_files = sorted(glob.glob(os.path.join(audio_folder,'validate', '*.wav')))

if load_subset is not None:
    train_files = train_files[:load_subset]
    val_files = val_files[:load_subset]

train_labels = {}
train_mel = {}
val_labels = {}
val_mel = {}

print('Founding scaler')
for n,id in enumerate(train_files):
    labels[id] = os.path.join(label_folder, 'train',os.path.basename(id).replace('.wav','.txt'))
for id in val_files:
    labels[id] = os.path.join(label_folder, 'validate',os.path.basename(id).replace('.wav','.txt'))

params['train'] = True
# Generators
print('Making training generator')
training_generator = DataGenerator(train_files, labels, **params)

params['train'] = False
params['sequence_hop_time'] = sequence_time # To calculate F1_1s

print('Making validation generator')
validation_generator = DataGenerator(val_files, labels, **params)

print('Getting data')

x_val,_,mel_val,_ = validation_generator.return_all()
x_train,_,mel_train,_ = training_generator.return_all()

scaler = Scaler(normalizer=normalize_data)

scaler.fit(mel_train)

mel_train = scaler.transform(mel_train)
mel_val = scaler.transform(mel_val)

Founding scaler
Making training generator
Making validation generator
Getting data
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %


In [19]:
print('\nBuilding model...')

sequence_samples = int(sequence_time*sr)
scaler_vector = scaler.get_scaler()

model = SMel(mel_bands,sequence_frames,audio_win,audio_hop,scaler=scaler_vector)    

model.summary()

##### Init filters
filters = mel_filters(sr = sr,n_fft = n_fft,N_mels = mel_bands,fmin = 0,fmax = sr/2,htk=htk,file_name='filters.npy', plot=False)
filters = np.expand_dims(filters,axis=1)
W = [filters.T, np.zeros(filters.shape[0])]
model.layers[1].set_weights(W)

opt = optimizers.Adam(lr=learning_rate)

if resume:
    print('Loading best weights and resuming...')
    weights_best_file = os.path.join(expfolder, 'weights_best.hdf5')
    model.load_weights(weights_best_file)

# Fit model
print('\nFitting model...')

if resume:
    f1s_best = resume_f1_best

csv_logger = CSVLogger(os.path.join(expfolder, 'training.log'))

model.compile(loss='mean_squared_error',optimizer=opt)

history = model.fit(x=x_train, y=mel_train, batch_size=2*batch_size, #Borrar el 10!
                            epochs=epochs, verbose=fit_verbose,
                            validation_split=0.0,
                            shuffle=True,
                            callbacks=[csv_logger],
                            validation_data=(x_val,mel_val))

model.save_weights(os.path.join(expfolder, 'weights_best.hdf5'))



Building model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 44, 1024, 1)       0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 44, 64, 128)       131200    
_________________________________________________________________
lambda_13 (Lambda)           (None, 44, 64, 128)       0         
_________________________________________________________________
lambda_14 (Lambda)           (None, 44, 128)           0         
_________________________________________________________________
lambda_15 (Lambda)           (None, 44, 128)           0         
_________________________________________________________________
lambda_16 (Lambda)           (None, 44, 128)           0         
Total params: 131,200
Trainable params: 131,200
Non-trainable params: 0
___________________________________________________

Epoch 61/101
60000/60000 [==============================] - 55s 920us/step - loss: 5.4511e-04 - val_loss: 7.6630e-04
Epoch 62/101
60000/60000 [==============================] - 55s 921us/step - loss: 5.7268e-04 - val_loss: 8.2576e-04
Epoch 63/101
60000/60000 [==============================] - 55s 921us/step - loss: 5.6850e-04 - val_loss: 5.3118e-04
Epoch 64/101
60000/60000 [==============================] - 55s 921us/step - loss: 5.5498e-04 - val_loss: 9.8069e-04
Epoch 65/101
60000/60000 [==============================] - 55s 921us/step - loss: 6.3262e-04 - val_loss: 4.7773e-04
Epoch 66/101
60000/60000 [==============================] - 55s 919us/step - loss: 5.8946e-04 - val_loss: 3.8413e-04
Epoch 67/101
60000/60000 [==============================] - 55s 919us/step - loss: 5.2085e-04 - val_loss: 4.2595e-04
Epoch 68/101
60000/60000 [==============================] - 55s 920us/step - loss: 5.7199e-04 - val_loss: 5.6575e-04
Epoch 69/101
60000/60000 [==============================] - 55s 

<Figure size 432x288 with 0 Axes>